# **Assignment 2 - The Secret Keepers**

# Stream Cipher

**Stream Ciphers** are, as the name suggests, one type of encryption where the plaintext is treated as a stream of elements; in particular, each bit or byte is encrypted individually. Thanks to this behaviour, this method of crypyography tends to be lighter and more efficient, but it is not always the case. The general implementation requires a way to create a proper key stream: starting from a given key, also called seed, we need to apply an operation called *key expansion*, which is able to generate a *pseudo-random* sequence of boolean values we will use to actually encrypt the plaintext. 

In [1]:
from math import log2
from math import ceil
from operator import xor

In [2]:
# Useful functions

def integer_to_bits(integer, nbit=None):
    '''
    Given an integer, it generates the corresponding sequence of bits
    EX:  11 (0b1011) --> [1, 0, 1, 1]
    ----------
    integer: int,
        integer to convert into a list of bits,
    nbit: int, optional (default None)
        length of the output sequence of bits.
        If None the length is ceil(log2(integer)) 

    Return
    ------
    list of bools,
        output bit sequence
    '''
    
    if nbit is None:
        nbit = max(1, ceil(log2(integer + 1))) 
        
    bits = []
    for i in range(nbit):
        bits.append(bool((integer & (1 << i)) >> i))
    return bits[::-1]

def bytes_to_bits(bytestream):
    '''
    Given a byte string, it generates the corresponding sequence of bits
    EX:  b'a' (0x61) --> [0, 1, 1, 0, 0, 0, 0, 1]
    ----------
    bytestream: bytes,
        byte string to convert into a list of bits,

    Return
    ------
    list of bool,
        output bit sequence with lenght 8*len(bytestream)
    ''' 
    
    integer = int.from_bytes(bytestream, byteorder='big')
    bits = integer_to_bits(integer, nbit=8*len(bytestream))
    
    return bits

def bits_to_integer(bits):
    '''
    Given a list of bits its integer representation is generated
    EX: [1, 0, 1, 1] -->  11 (0b1011)
    ----------
    bits: list,
        the list of bits (each represented with an integer 0/1 or a bool)

    Return
    ------
    int,
        integer having the binary representation defined by the list
    '''
    
    integer = 0
    
    for bit in bits:
        integer = (integer << 1) ^ bit
        
    return integer

def bits_to_bytes(bits, num_bytes=None, byteorder='big'):
    '''
    Converts a list of bits into bytes
    EX: [1, 0, 1, 1] -->  11 (0b1011)
    ----------
    bits: list,
        the list of bits (each represented with an integer 0/1 or a bool)
    num_bytes: int, optional
        the number of bytes to use for the representation
        (when None the least ammout of bytes is used)  
    byteorder: str, optional
        the order of the bytes
        (defualt is big)

    Return
    ------
    bytes,
        byte string defined by the integer
    '''
    
    if num_bytes is None:
        num_bytes = ceil(len(bits) / 8)
        
    bytestream = bits_to_integer(bits).to_bytes(num_bytes, byteorder)
    return bytestream


def bxor(b1, b2):
    '''
    XOR operation between bytes
    ----------
    b1, b2: byte,
        couple of bytes that we want to xor

    Return
    ------
    byte,
        resulting byte from the operation
    '''

    result = b""

    for b1, b2 in zip(b1, b2):
        result += bytes([b1 ^ b2])

    return result

## LFSR

One possible, and deterministic, solution to the creation of a proper sequence of elements with the highest possible incorrelation, is given by a model called **LFSR**, which stands for *Linear Feedback Shift Register*: this model belongs to the so called group of **CPRNG**s, *Criptographically-Secure Pseudo-Random Number Generators*, since it's output is in a certain way "unpredictable" from the outside and, once known an already created finite sequence of values, it's computationally unfeasible to predict the following one. 

Let's briefly look at how an LFSR works: given a key in the form of a polynomial of degree *m* and an initial state made of *m* elements (assuming we are working with an irreducible polynomial, also defined *primitive*), at each cycle: 
1. We compute the *feedback bit* as the XOR of all the AND operations made between the respective elements of the <span style="color:lightblue">polynomial's coefficients list</span> and the <span style="color:lightgreen">current state list</span>;
2. We shift to the right the state list inserting the *feedback bit* as the new least significant bit of the state list;
3. We output the <span style="color:orange">most significant bit</span> of the state, S0.

With the iteration of this algorithm we get a list of boolean values, which can be used as the stream key to our system.



![LFSR](LFSR2.jpg)

In [3]:
class LFSR():
    ''' 
    self.counter: int,
        needed to take trace of the number of cycles executed
    self.state: list,
        list of integrers representing the current state of the LFSR [Sm-1, ..., S2, S1, S0]
    self.poly: list,
        list of integers representing the coefficients of the LFSR's polynomial function in an increasing order [p1, p2, p3, ..., pm]
    self.length: int,
        maximum degree of the polynomial, equal to the length of the state list
    self.output: int,
        most significant bit of the state list: S0
    self.feedback: int,
        integer representing the boolean value computed by the LFSR at each cycle, starting from it's current state and
        the poly list
    '''
    
    def __init__(self, poly, state=None):
        ''' 
        Initialization of the LFSR
        ---------
        poly: list,
            list of integers in descending order (EX: [3,1,0]), where each number represents which degree of the polynomial
            has coefficient equal to 1
        state: bin,
            binary value (EX: 0b111) which represents the list of boolean values of the initial state ... S2, S1, S0
        '''
        
        self.counter = 0
        self.state = []
        self.poly = []
        self.length = poly[0] # Equal to the degree of the polynomial

        if (state == 0):
            raise ValueError('The initial state must be different from zero, try with another initial state!')

        # If the state is not given, as default we put all 1s as the initial state
        # If it's given, we translate the binary '0bxx...' into a list of integers 0s and 1s
        if state is None:
            [self.state.append(1) for x in range(self.length)]
        else:
            [self.state.append(int(x)) for x in bin(state)[2:]]

        for i in range(self.length):
            if (len(self.state) < self.length):
                self.state = [0] + self.state
                
        self.output = self.state[-1] 

        # I need a list made of 0s and 1s: for a polynomial like x^3 + x + 1, given to the class as [3,1,0],
        # I get a list like [1,0,1], corresponding to 1x + 0x^2 + 1x^3
        [self.poly.append(0) for x in range(self.length)]
        if poly[-1] == 0: poly.remove(0) 
        for x in poly:
            self.poly[x-1] = 1

        self.feedback = 0
        for i, bit in enumerate(self.poly):
            self.feedback = xor(self.feedback, bit and self.state[i])
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        ''' 
        We run one cycle of the LFSR, in a sequential way
        ---------

        Return
        ---------
        int,
            output generated by the LSFR at each cycle, equal to the most significant bit of the state list
        '''
        
        self.counter += 1

        self.state = [self.feedback] + self.state[:-1] # State update: shift to the right and feedback bit injected on the left

        self.feedback = 0
        for i, bit in enumerate(self.poly):
            self.feedback = xor(self.feedback, bit and self.state[i])

        self.output = self.state[-1]

        return self.output
 
    def cycle(self):
        ''' 
        We perform a complete cycle of the given LFSR
        ---------

        Return
        ---------
        list,
            list of integers representing boolean values outputted by the LSFR
        '''
        
        self.counter = 0
        list_of_bool = []

        for x in self:
            if (self.counter <= ((2**self.length) - 1)):
                list_of_bool.append(x)
            else:
                break
                
        return list_of_bool
    
    def run_steps(self, N=1):
        ''' 
        We run the LSFR for a number N of times
        ---------
        N: int,
            number of cycles required

        Return
        ---------
        list,
            list of integers representing boolean values outputted by the LSFR
        '''
    
        self.counter = 0
        list_of_bool = []

        for x in self:
            if (self.counter <= N):
                list_of_bool.append(x)
            else:
                break

        return list_of_bool
    
    def __str__(self):
        ''' 
        Useful to quickly see the current list of the attributes of the LFSR
        '''

        return f'Poly: {self.poly}, Current state: {self.state}, Output bit: {self.output}, Feedback bit: {self.feedback}'

### Testing

In order to test the functionality of the class defined above we need first of all to create different LFSRs; once created, we want to test all possible methods on them, looking for errors and misinterpretation of input data.

For the first LFSR is simple to see whether mistakes are present, since it's simple to compute the correct values by hand. For the other two I will compute manually the first values of the correct sequences and then compare them.

Prove class functionality (test every method) in these three cases:
- `poly = [3, 1, 0]` and `state = 0b110`
- `poly = [5, 2, 1, 0]` and `state = 0b00101`
- `poly = [9, 5, 0]` and `state = 0b111111111`

In [4]:
lfsr1 = LFSR([3,1,0], 0b110)
lfsr2 = LFSR([5,2,1,0], 0b00101)
lfsr3 = LFSR([9,5,0], 0b111111111)

# __next__ and __str__ test:
print(lfsr1.cycle())
print(next(lfsr1))
print(next(lfsr1))
print(next(lfsr1))
print(lfsr1)

print()
# run_steps(N) test:
print(lfsr2.cycle())
print(lfsr2.run_steps(5))
print(lfsr2.run_steps(2))
print(lfsr2)

print()
# cycle() test:
print(lfsr3.cycle())
print(lfsr3)

[1, 1, 1, 0, 1, 0, 0]
1
1
0
Poly: [1, 0, 1], Current state: [0, 1, 0], Output bit: 0, Feedback bit: 0

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0]
[1, 1, 1, 0, 1]
[0, 1]
Poly: [1, 1, 0, 0, 1], Current state: [0, 1, 0, 1, 1], Output bit: 1, Feedback bit: 0

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

All methods seem to behave correctly: next function cycles the LFSR once and outputs the new bit, sequentially; run_steps(N) runs the asked number of cycles and, if called again, starts the iteration from where it was left; and finally cycle(), as it should, runs exactly (2^length - 1) steps, equal to the period of the considered LSFR, and outputs the total sequence generated.

From this data we can actually see that the second LFSR proposed is equal to the first one, since it outputs the same sequence of bits, but it isn't a primite LFSR.

## Berlekamp Massey

**Berlecamp Massey (BM) algorithm** is one of the reasons why LFSRs are not widely used in cryptography: thanks to this algorithm any attacker can perform a *kwown plaintext attack* (KPA) successfully, and retrieve the whole polynomial function P(x) related to the shortest LFSR. The idea behind the method is the following:

1. We first need to known a sequence of bits b[0], b[1], b[2], ..., b[n];
2. Using the concept of *discrepancy*, defined as the **XOR**, **from i=0 to n**, of **Pi AND b[t-i]**, we can obtain a linear system of equations with Pi as unknowns, the coefficients of the polynomial.

This of course can be done because of the reversibility of the XOR function and since the sequence of bits created by any LFSR actually contains every state of it, including the initial one.

In [5]:
def berlekamp_massey(bits):
    ''' 
    Algorithm able to retrieve the shortest LFSR from a sequence of bits of a certain length, generated by the LFSR itself
    ---------
    bits: list,
        list of integers representing boolean values, generated as output by the LFSR we want to study

    Return
    ---------
    list,
        list of integers representing the polynomial function of the LFSR, formatted as in the example: [3,1,0] = x^3 + x + 1
    '''
    
    m = 0
    r = 1
    N = len(bits)
    P = [1] # P(x) = 1
    Q = [1] # Q(x) = 1
    R = []

    for t in range(N):
        d = 0
        for j in range(m+1):
            d = xor(d, P[j] and bits[t-j]) # Discrepancy computation
        if (d == 1):
            Qtemp = Q.copy()        #
            for i in range(r):      #
                Qtemp = [0] + Qtemp # Qtemp(x) = Q(x)*(x^r)
            if (2*m <= t):
                R = P.copy() # R(x) = P(x)
                for i in range(min(len(Qtemp), len(P))): #      
                    P[i] = xor(P[i], Qtemp[i])           #
                if (len(Qtemp) > len(P)):                #
                    P += Qtemp[len(P):]                  # P(x) = P(x) + Qtemp(x)  
                Q = R.copy() # Q(x) = R(x)
                m = t + 1 - m
                r = 0
            else:
                for i in range(min(len(Qtemp), len(P))): # 
                    P[i] = xor(P[i], Qtemp[i])           #
                if (len(Qtemp) > len(P)):                #
                    P += Qtemp[len(P):]                  # P(x) = P(x) + Qtemp(x)
        r += 1

    # I need a list made of integers which tell me where in the polynomial I have a coefficient 1 or 0: 
    # for a polynomial like 1 + 1x + 0x^2 + 1x^3, given by the algorithm as [1,1,0,1],
    # I get a list like [3,1,0], corresponding to x^3 + x + 1
    poly = []
    for i, x in enumerate(P):
        if (x):
            poly = [i] + poly

    return poly

### Testing

To test the BM method we need to have correct sequences of bits, created by existing LFSR models; to do so we can take advantage of the test carried out previously, the one related to the LFSR class, in order to have the complete output of each model. Once the lists of bits are ready, we input them to the function we need to test, the Berlekamp Massey method, and we print the obtained polynomials. The only thing left to do is to compare them to the correct ones, in order to verify the behaviour of the implemented algorithm.

We want to prove its functionality in these three cases, using the already defined LFSR class:
- `poly = [3, 1, 0]` and `state = 0b110`
- `poly = [5, 2, 1, 0]` and `state = 0b00101`
- `poly = [9, 5, 0]` and `state = 0b111111111`

In [6]:
bits1 = [1, 1, 1, 0, 1, 0, 0]
bits2 = [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0]
bits3 = [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]

print(f'The polynomial corresponding to the first sequence is: {berlekamp_massey(bits1)}')
print(f'The polynomial corresponding to the second sequence is: {berlekamp_massey(bits2)}')
print(f'The polynomial corresponding to the third sequence is: {berlekamp_massey(bits3)}')

The polynomial corresponding to the first sequence is: [3, 1, 0]
The polynomial corresponding to the second sequence is: [3, 1, 0]
The polynomial corresponding to the third sequence is: [9, 5, 0]


As hinted during the first testing, the first and the second LFSRs share the same output pattern, so they can be represented with the same primitive polynomial, which means they have the same *linear complexity*.

Given the following key stream `keystream`, generated by an LFSR, apply the Berlekamp-Massey Algorithm to discover the structure (in particular, the polynomial) of the LFSR.

In [7]:
keystream = b'\xfa\xb3v\x93\x8b\xca0\x83'

keystream_bits = bytes_to_bits(keystream)
for i, x in enumerate(keystream_bits): #
    keystream_bits[i] = int(x)         # I don't like working graphically with True/False, I prefer 1/0 

# BM algorithm:
print(f'Keystream: {keystream_bits}')
print(f'The corresponding LFSR\'s polynomial is: {berlekamp_massey(keystream_bits)}')

Keystream: [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]
The corresponding LFSR's polynomial is: [6, 1, 0]


## LFSR-based Generator

There exist many families of application based on the LFSR structure, and one of these is called **Alternating Step Generator**.

Its functioning is simple once we look at its schematic: we take the clock signal as the input of the first LFSR, called <span style="color:lightgreen">controller LFSR</span>, whose job will be to decide which of the two LFSRs downstream has to be clocked. The output of the controller divides into two lines, one with its true value and one with its negation, and the two become the inputs of two AND gates; the other couple of inputs are the clock itself. Finally the output of the two ANDs enters two different blocks, <span style="color:orange">LFSR 0</span> and <span style="color:orange">LFSR 1</span>, whose outputs get xored to create the actual output of the whole device, the <span style="color:lightblue">stream s</span>.

![Alternating Step Generator](AltStepGen2.jpg)

- `poly0 = [5, 2, 0]`
- `poly1 = [3, 1, 0]`
- `polyC = [2, 1, 0]`

In [8]:
class AlternatingStepGenerator():
    ''' 
    self.seed: int,
        seed that will be split into three parts, each used as the initial state for one of the three LFSRs of the device,
        for this reason we need to have a seed of exactly 10 bits for this ASG
    self.LFSRc: LFSR,
        controller, responsible for the clocking of the two other LFSRs
    self.LFSR0, self.LFSR1: LFSR,
        devices whose outputs combination generates the stream
    self.outc: int,
        integer value representing the boolean output of the controller
    self.ou0, self.out1: int,
        integer values representing the boolean outputs of the respective LFSR
    '''
    
    def __init__(self, seed):
        ''' 
        Initialization of the Alternating Step Generator
        ---------
        seed: int,
            integer representing a binary value (EX: 0b111) which will be used as the initial state for the three LFSRs
        '''
        
        self.seed = seed
        if (self.seed == 0):
            raise ValueError('The seed must be different from zero, try with another initial state!')
        
        seed_c = bin(self.seed)[0:2] + bin(self.seed)[10:12] # We split the seed into 3 keys,
        seed_0 = bin(self.seed)[0:7]                         # one per each LFSR of the ASG
        seed_1 = bin(self.seed)[0:2] + bin(self.seed)[7:10]  #

        self.LFSRc = LFSR([2,1,0], int(seed_c, 2))
        self.LFSR0 = LFSR([5,2,0], int(seed_0, 2))
        self.LFSR1 = LFSR([3,1,0], int(seed_1, 2))

        self.outc = 0
        self.out0 = int(seed_0[-1], 2)
        self.out1 = int(seed_1[-1], 2)
        
    def __iter__(self):

        return self
    
    def __next__(self):
        ''' 
        We run one cycle of the Alternating Step Generator, in a sequential way
        ---------

        Return
        ---------
        int,
            output generated by the ASG at each iteration, equal to the XOR operation between the outputs of LFSR0 and LFSR1
        '''
        
        self.outc = next(self.LFSRc)
        if (not self.outc):
            self.out0 = next(self.LFSR0) # Sample a new bit from LFSR0
        elif (self.outc):
            self.out1 = next(self.LFSR1) # Sample a new bit from LFSR1

        bit = xor(self.out0, self.out1) # out0 XOR out1
        
        return bit

Compute the Linear complexity of the generated sequence.

In [9]:
seed = 0b1101110101
stream = []
print(f'Seed: {seed}')

ASG = AlternatingStepGenerator(seed)
for i in range(seed):
    stream.append(next(ASG))

print(f'Produced bit stream: {stream}')

# BM algorithm:
lin_com = berlekamp_massey(stream)
print(f'The corresponding LFSR\'s polynomial is: {lin_com}, which means that its linear complexity is: {lin_com[0]}')

Seed: 885
Produced bit stream: [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,

To compute the *linear complexity* of the generated ASG, we need to find the characteristic polynomial with the highest degree possible corrisponding to a LFSR with the same complexity (degree); to do this we try to input different seeds until we find a point where the polynomial, computed using the BM algorithm, doesn't change any further.

In this particular case we discover that this Alternating Step Generator has a linear complexity equal to the one of a LFSR of degree 24.

### Testing

In general, any bit sequence generator able to create a keystream starting from a given key or seed can be implemented to create a stream cipher system; indeed its implementation is mandatory if we want to realize the process of *key expansion*. Once the keystream is created and delivered to our system, what we need to do is to XOR, bit to bit or also byte to byte, one element of the expanded key with one element of the plaintext, when encrypting, or of the ciphertext, when decrypting, in a sequential manner.

Given the key `key`, decrypt the ciphertext `ciphertext`.

In [10]:
key = 0x3FF # integer value
ciphertext = (
    b'Y\xea\xfc\xc2\x8c\x17p{\x1f8\xbf,N\\\xf8\x97\xeb\x99#\'#\xf3\x1cY\xfd'
    b'\x82\xe9\xbe\xc2\xeb\x16H\xd0Q\xd5\xa8Y\x8e\x8b\\\xeb\x8d\xe1\xea\xf5'
    b'\x83\xb0\xe7\xee\xf2\\\xear\x848l\xe2\xb2\x06ov%\xdb\x08\x13\xf2qy\xf6'
    b'}\xfdO\xebG{\xaa\xaf\xbaP\xc1\x98c\x19\xdc5\xf3\x00P_\x0e\x8b\xe9\xa5$'
    b'\xaf,7\x99G\xe5\x89f\xd5o\xd9s\xd4\x10;\xf4\x10\x1a\x84]\xf6>\xd9J\x86'
    b'\xb2/%\x86\x92\xdc3f\x1d\x15\xa0\xa7K1*\xa0\xaa\x88+x\xb9\xa9#Ie)\xf97'
    b'\x05\xf1\x1b\x02~\xeac\xd0\xeb\x0bv+\xd2Y\xb1\xbd\x1d6!\xde\xd9\x9de'
    b'\x05\xb3\xf30\x14\xfd\xa7s\xa9\xce\xad]\x01W\x0b\x9a\xb6\x03\x8e^J\xbf'
    b'\x01\xe4\xb6\xa0\x83\xbej\xb6\xda\xca\xa3J\xd78~1\xaf/t\x1eL\x93\x19J'
    b'\xb18Bm\x7f\xddZ\xa8L\xbd\xb1\x84,\x04\xc7\x0b,\x11\xce}\xed\x06\x06XZ'
    b'\xaf\xcf\xaa-\xaf\xad"\x87\xd3\xf8\xc2\xf7\xd3\x1bM}\xb9\x00)\xb1\x9a'
    b'\x06\xcdU\xedV\xd7\x03\x90\xed;>\xc5\xd7\xff\xa0qZ\x94\xf3\xb6\x1b9v'
    b'\xfa\xfa\xf1x}\xd9\xf3\x7fm\xe4 \xe0"G\xe0O*C%p\xd7yYS&\xd9{\xec\xe9'
    b'=46\xfbH\xcc#\x0f\xe8\xf78H\xcc *\xb8\xd8\xe35\xda\x03>\xc5\xf0\x1a'
    b'OCZ\xfc\x11\xbd\xf7\xb0\xc9\xb2!\xfe\xd8\xc7\x8e\x1c\xc3:\x7fb\xdd9wZ'
    b'\xad\xca\\\x83\xf9>Fx\x1dQ\x1d\x9a\x92\xdb\xc1\x8b+\x19\xdfDK\x93\xd7M'
    b'\xe7Cg\xdbP\xa6\x99\xe5`\xae\xed6E\xcf\xe3\xc2\xb5\xee\x80\x14D+@5\xb2'
    b'\xde\x02\xdb\x01\x9b\xd9\x90<\x00\xe6\n=\x98\xf6\xe9\xb7\x14\x93\x95'
    b'\xc8\xf7YX!\xe2\x830<q\x9b\xed\x034\xa0\x0c|(\x05%h3\x87dN\x160zN\''
    b'\x8ev\xe4\xe0\xb0q\x02\xb1\x10\xa0\x90\x06\xf42SSV4nl\xf4\xd8\xe1\xc3S'
    b'?\x89\xe5\x80\x11X\x1f\xfe-"\xed\xb4D\xb6a\xa3\xdd\xc8\xca\t\xfcrg\x0e'
    b'\xfa|X\x16\x82\xc2\xdb\x86\xfd=\x07cK\x15?\x98\xd3\xf8\xda\xcb\x0c\x0e'
    b'\x84\\\x9c\x84\x87\xd1\xa5P\xab\xcd;'
)

In [11]:
print(f'Key: {key} = {bin(key)}')

ciphertext_b = bytes_to_bits(ciphertext)
print(f'Ciphertext: {ciphertext}')

keystream = []
plaintext = b""

ASG = AlternatingStepGenerator(key)
for i in range(len(ciphertext_b)):
    keystream.append(next(ASG))

keystream_bytes = bits_to_bytes(keystream)

for x, y in zip(ciphertext, keystream_bytes):
    plaintext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))
    
print(f'Decrypted Plaintext: {bytes(plaintext)}')

Key: 1023 = 0b1111111111
Ciphertext: b'Y\xea\xfc\xc2\x8c\x17p{\x1f8\xbf,N\\\xf8\x97\xeb\x99#\'#\xf3\x1cY\xfd\x82\xe9\xbe\xc2\xeb\x16H\xd0Q\xd5\xa8Y\x8e\x8b\\\xeb\x8d\xe1\xea\xf5\x83\xb0\xe7\xee\xf2\\\xear\x848l\xe2\xb2\x06ov%\xdb\x08\x13\xf2qy\xf6}\xfdO\xebG{\xaa\xaf\xbaP\xc1\x98c\x19\xdc5\xf3\x00P_\x0e\x8b\xe9\xa5$\xaf,7\x99G\xe5\x89f\xd5o\xd9s\xd4\x10;\xf4\x10\x1a\x84]\xf6>\xd9J\x86\xb2/%\x86\x92\xdc3f\x1d\x15\xa0\xa7K1*\xa0\xaa\x88+x\xb9\xa9#Ie)\xf97\x05\xf1\x1b\x02~\xeac\xd0\xeb\x0bv+\xd2Y\xb1\xbd\x1d6!\xde\xd9\x9de\x05\xb3\xf30\x14\xfd\xa7s\xa9\xce\xad]\x01W\x0b\x9a\xb6\x03\x8e^J\xbf\x01\xe4\xb6\xa0\x83\xbej\xb6\xda\xca\xa3J\xd78~1\xaf/t\x1eL\x93\x19J\xb18Bm\x7f\xddZ\xa8L\xbd\xb1\x84,\x04\xc7\x0b,\x11\xce}\xed\x06\x06XZ\xaf\xcf\xaa-\xaf\xad"\x87\xd3\xf8\xc2\xf7\xd3\x1bM}\xb9\x00)\xb1\x9a\x06\xcdU\xedV\xd7\x03\x90\xed;>\xc5\xd7\xff\xa0qZ\x94\xf3\xb6\x1b9v\xfa\xfa\xf1x}\xd9\xf3\x7fm\xe4 \xe0"G\xe0O*C%p\xd7yYS&\xd9{\xec\xe9=46\xfbH\xcc#\x0f\xe8\xf78H\xcc *\xb8\xd8\xe35\xda\x03>\xc5\x

## RC4

**Rivest Cipher 4**, denoted as **RC4**, is another stream cipher method that has been presented in 1987. As every stream cipher, its functioning requires a unique way of creating a keystream which has to be used to encrypt the plaintext; in this case the keystream is generated from a secret internal state, which itself is composed by two different parts: a permutation P of all 256 possible bytes and two 8-bit index-pointers i and j. The permutation is initialized and managed through a *Key Scheduling Algorithm* (KSA), which takes as input a key of length L: once that each P[i] is initialized with the value of i, the algorithm proceeds to mix the permutation's values in relation to the key. Finally, to generate the needed keystream, we rely on a *Psuedo-Random Generation Algorithm* (PRGA), which modifies the state and returns a byte as output that can be used for the encryption. 

In [12]:
class RC4():
    
    def __init__(self, seed, drop=0):
        '''
        self.seed: byte,
            key used to initialize and manage the permutation P (EX: b'\xb1\xd7...')
        self.drop: int,
            number of bytes that we have to discard before outputting useful values
        self.P: list,
            permutation byte list, fundamental element of the total state vector [P + i + j],
            made of (256 + 1 + 1) bytes
        self.length: int,
            length of the seed given as input, which is the number of bytes the key is made of (5 <= length <= 16)
        self.i, self.j: int,
            integer values used as indices during the whole process and parts of the total state vector   
        '''
        
        self.counter = 0
        self.seed = seed
        self.drop = drop
        self.P = []
        self.i = 0
        self.j = 0
        self.length = len(self.seed)

        for self.i in range(256): # P initialization
            self.P.append(self.i) 
            self.P[self.i] = self.P[self.i].to_bytes(1, 'big')

        for self.i in range(256):
            self.j = (self.j + int.from_bytes(self.P[self.i], "big") + self.seed[self.i % self.length]) % 256
            self.P[self.i], self.P[self.j] = self.P[self.j], self.P[self.i]

        self.i = 0
        self.j = 0
    
    def __iter__(self):

        return self
    
    def __next__(self):
        '''
        We run one cycle of the RC4, in a sequential way
        ---------

        Return
        ---------
        byte,
            byte output generated by the RC4 at each iteration
        '''
        
        self.counter += 1

        self.i = (self.i + 1) % 256
        self.j = (self.j + int.from_bytes(self.P[self.i], "big")) % 256
        self.P[self.i], self.P[self.j] = self.P[self.j], self.P[self.i]

        byte = self.P[(int.from_bytes(self.P[self.i], "big") + int.from_bytes(self.P[self.j], "big")) % 256]

        return byte
        
    def run_with_drop(self, N=1):
        '''
        We run the RC4 for a number of times equal to (self.drop + N),
        method to use when we want to create a whole keystream of N bytes 
        discarding the first self.drop bytes
        ---------
        N: int,
            number of bytes needed

        Return
        ---------
        byte,
            byte sequence made by appending the byte values obtained at each next iteration
        '''

        self.counter = 0
        byte_stream = b""
        
        for num in range(self.drop): # Drop the first elements
            self.__next__()

        for x in range(N):
            byte_stream += self.__next__()

        return byte_stream

### Testing

The possible employment of an RC4 as a stream cipher is similar to the one explained for the ASG: the RC4 structure is the part of the whole system required to expand the initial key into a keystream; the keystream is then xored to the plaintext to encrypt it, or to the ciphertext to decrypt it. The only difference that occur in this implementation regards the dimension of the value created by the number generator at each iteration: in this case the keystream is made of bytes (8 bits each), while any LFSR-based number generator outputs simply bits.

Given the key `key`, decrypt the ciphertext `ciphertext`.

In [13]:
key = b'0123456789ABCDEF' # integer value
ciphertext = (
    b"\x0f\x9e\xec\xc3\x0f\xeck\xa5\xc9\xd4\xd4\xb81\xf0\xd5[\x93"
    b"\xbc\xdaY^\x0e\xc1'\xdb\x1eP\xc0uD\xeb\x91E]\x15\xcf\x85\x07%"
    b"\x97\xffl\xf1\xbb\xe1*\xa0\xee\xd3\x94\x17\xb0|e\x93h\xd7\xc5"
    b"\xc3tT\x84\xee%\xcb\x15\r\xc4\xe8\n\xf2\xd0.\xec\xc6\xe1-\xb4"
    b"\x8b\x9a\x14\x823k\xab?\x8b\x9c\xaas\xa1#\xb8\xb2\xceh\xb5\x8b"
    b"'\x90B}C\x80~\x8cr\xde\xc9\xe2\x17\xe45\xb9\x10\x94\xd4\x0eRJ"
    b"\x0fr&\xe7\xe3P\xbfz\xecIA\x94\xe60\xa8{_\x03\xc7\x91\xcf\xc6"
    b"\x04\xfc\x8d\x86-E\x13\xba\x13i\x17;\xd7\x8e \xa5\xe6\xa5uR\n"
    b"89z\xe2YZ!e\x0f,s\x9a\xacN\xab\xc7\xcaOO\x81\xe06\x03\xac7\x9b"
    b",%\xf7\x9d(\xde\x0b\xc3\xbf\xbe7\xc7<\xf4r\x0eLz\xd8\xe5b\xa0n"
    b"\x11f\xfe\xca+RNnL\xef+\x1b\x1c\x0b\x8a\xb6M\xbdU\xd6\x1c\x9cn"
    b"\t\x8aX>B9\xa4\xf7\xd7jS\xa3\xfd\xeb=\xeeY\xbf\x8dG\xab\xc6"
    b"\x08 \xfba\x90\xdbla\xbf\xf0\x9c\x1a\xa0\xcb\x9a\x7f\x88X\x1dV"
    b"\xd2'\xba\x1b\x16l\xc3vx\xddqU\xf72@\x86>A\xe6b\x050Y\xed\x8bJ"
    b"!\xe0\x80I\xa4X\x9e\xd6^\x126\x99\x93\xa3(\xc5\xf0\xbd\xdei\xcb"
    b"\xdc5\xf58\xda\xa5\x96\xb9!5>\xdbJ\xd4\xef\xe1\x0e\xfeo\x97`"
    b"\xd7[\x18\t\xf4/DV\xdd\xe8~#\xbd\xfd{B\xacC=\xa9\xd5\xbb\xdbH"
)
drop = 3072

In [14]:
print(f'Key: {key}')
print(f'Ciphertext: {ciphertext}')
plaintext = b""

rc4 = RC4(key, drop=drop)

for x, y in zip(ciphertext, rc4.run_with_drop(len(ciphertext))):
    plaintext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

print(f'Decrypted Plaintext: {bytes(plaintext)}')


Key: b'0123456789ABCDEF'
Ciphertext: b"\x0f\x9e\xec\xc3\x0f\xeck\xa5\xc9\xd4\xd4\xb81\xf0\xd5[\x93\xbc\xdaY^\x0e\xc1'\xdb\x1eP\xc0uD\xeb\x91E]\x15\xcf\x85\x07%\x97\xffl\xf1\xbb\xe1*\xa0\xee\xd3\x94\x17\xb0|e\x93h\xd7\xc5\xc3tT\x84\xee%\xcb\x15\r\xc4\xe8\n\xf2\xd0.\xec\xc6\xe1-\xb4\x8b\x9a\x14\x823k\xab?\x8b\x9c\xaas\xa1#\xb8\xb2\xceh\xb5\x8b'\x90B}C\x80~\x8cr\xde\xc9\xe2\x17\xe45\xb9\x10\x94\xd4\x0eRJ\x0fr&\xe7\xe3P\xbfz\xecIA\x94\xe60\xa8{_\x03\xc7\x91\xcf\xc6\x04\xfc\x8d\x86-E\x13\xba\x13i\x17;\xd7\x8e \xa5\xe6\xa5uR\n89z\xe2YZ!e\x0f,s\x9a\xacN\xab\xc7\xcaOO\x81\xe06\x03\xac7\x9b,%\xf7\x9d(\xde\x0b\xc3\xbf\xbe7\xc7<\xf4r\x0eLz\xd8\xe5b\xa0n\x11f\xfe\xca+RNnL\xef+\x1b\x1c\x0b\x8a\xb6M\xbdU\xd6\x1c\x9cn\t\x8aX>B9\xa4\xf7\xd7jS\xa3\xfd\xeb=\xeeY\xbf\x8dG\xab\xc6\x08 \xfba\x90\xdbla\xbf\xf0\x9c\x1a\xa0\xcb\x9a\x7f\x88X\x1dV\xd2'\xba\x1b\x16l\xc3vx\xddqU\xf72@\x86>A\xe6b\x050Y\xed\x8bJ!\xe0\x80I\xa4X\x9e\xd6^\x126\x99\x93\xa3(\xc5\xf0\xbd\xdei\xcb\xdc5\xf58\xda\xa5\x96\xb9!5>\xdbJ\xd4\xef

## Bonus Task

Build an object implementing a Stream Cipher that, given a Pseudo Random Number Generator (PRNG) and a key, can encrypt and decrypt a message. Note that, PRNG can output either a bit (LFSR-based generator) or a byte (RC4).

In [47]:
class StreamCipher():

    def __init__(self, key, prng, **kwargs):
        ''' 
        Initialization of a generic Stream Cipher
        ---------
        key: bytes,
            seed in the form of bytes that will be used to initialize all possible PRNGs
        prng: str,
            string of text that specify the required type of number generator ('lfsr', 'asg' or 'rc4'),
            of course for each possible PRNG the user needs to input all the required values as arguments
        kwargs: dict,
            dictionary that possibly contains the needed values:
            for an 'rc4' it will contain the drop value,
            for an 'lfsr' it will contain the polynomial.
        '''
   
        self.prng = prng
        self.key = key

        self.args = [key for key in kwargs.keys()]
        self.values = [value for value in kwargs.values()]
        
        if (self.args and self.args[0] == 'drop'): # We get the drop value for the RC4 
            self.drop = self.values[0]
        elif (self.args and self.args[0] == 'poly'): # We get the polynomial values for the LFSR
            self.poly = self.values[0]
    
    def encrypt(self, plaintext):
        '''
        We initialize the required type of random number generator, with the needed inputs and formats;
        then we run the encryption, always in a byte to byte manner.
        ---------
        plaintext: bytes,
            plaintext encoded in bytes, with an utf-8 encoding

        Return
        ---------
        ciphertext,
            byte sequence of the encrypted plaintext, which depends on the used PRNG
        '''
    
        ciphertext = b""
    
        if (self.prng == 'rc4'): # RC4, byte-based PRNG
            rc4 = RC4(self.key, drop=self.drop)

            for x, y in zip(plaintext, rc4.run_with_drop(len(plaintext))):
                ciphertext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        elif (self.prng == 'asg'): # ASG, bit-based PRNG
            asg = AlternatingStepGenerator(bits_to_integer(bytes_to_bits(self.key)))
            keystream = []

            plaintext_bits = bytes_to_bits(plaintext)

            for i in range(len(plaintext_bits)):
                keystream.append(next(asg))

            keystream_bytes = bits_to_bytes(keystream)

            for x, y in zip(plaintext, keystream_bytes):
                ciphertext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        elif (self.prng == 'lfsr'): # LFSR, bit-based PRNG
            lfsr = LFSR(self.poly, bits_to_integer(bytes_to_bits(self.key)))
            keystream = []

            plaintext_bits = bytes_to_bits(plaintext)

            for i in range(len(plaintext_bits)):
                keystream.append(next(lfsr))

            keystream_bytes = bits_to_bytes(keystream)

            for x, y in zip(plaintext, keystream_bytes):
                ciphertext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        return ciphertext

    def decrypt(self, ciphertext):
        '''
        We initialize the required type of random number generator, with the needed inputs and formats;
        then we run the decryption, always in a byte to byte manner.
        ---------
        ciphertext: bytes,
            ciphertext encrypted in bytes with one of the three possible methods of encryption

        Return
        ---------
        plaintext,
            byte sequence of the decrypted ciphertext
        '''

        plaintext = b""
    
        if (self.prng == 'rc4'): # RC4, byte-based PRNG
            rc4 = RC4(self.key, drop=self.drop)
            
            for x, y in zip(ciphertext, rc4.run_with_drop(len(ciphertext))):
                plaintext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        elif (self.prng == 'asg'): # ASG, bit-based PRNG
            asg = AlternatingStepGenerator(bits_to_integer(bytes_to_bits(self.key)))
            keystream = []

            ciphertext_bits = bytes_to_bits(ciphertext)

            for i in range(len(ciphertext_bits)):
                keystream.append(next(asg))

            keystream_bytes = bits_to_bytes(keystream)

            for x, y in zip(ciphertext, keystream_bytes):
                plaintext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        elif (self.prng == 'lfsr'): # LFSR, bit-based PRNG
            lfsr = LFSR(self.poly, bits_to_integer(bytes_to_bits(self.key)))
            keystream = []

            ciphertext_bits = bytes_to_bits(ciphertext)

            for i in range(len(ciphertext_bits)):
                keystream.append(next(lfsr))

            keystream_bytes = bits_to_bytes(keystream)

            for x, y in zip(ciphertext, keystream_bytes):
                plaintext += bxor(x.to_bytes(1, "big"), y.to_bytes(1, "big"))

        return plaintext

### Testing

To completely test the Stream Cipher class, we employ all the three possible PRNGs, always encrypting and decrypting the same initial plaintext:

In [49]:
message = 'hello world!'
key = b'12345678' # 0x12345678
print(f'Key: {key}')

plaintextA = message.encode('utf-8') # string to bytes
print(f'Plaintext: {bytes(plaintextA)}')

print()
# RC4 example -------------------------------------
alice = StreamCipher(key, 'rc4', drop=3072) 
bob   = StreamCipher(key, 'rc4', drop=3072)

ciphertext = alice.encrypt(plaintextA) # encryption by Alice
print(f'RC4 Encrypted Ciphertext: {bytes(ciphertext)}')

plaintextB = bob.decrypt(ciphertext) # decryption by Bob
print(f'RC4 Decrypted Plaintext: {bytes(plaintextB)}')

print()
# ASG example -------------------------------------
alice = StreamCipher(key, 'asg') 
bob   = StreamCipher(key, 'asg')

ciphertext = alice.encrypt(plaintextA) # encryption by Alice
print(f'ASG Encrypted Ciphertext: {bytes(ciphertext)}')

plaintextB = bob.decrypt(ciphertext) # decryption by Bob
print(f'ASG Decrypted Plaintext: {bytes(plaintextB)}')

print()
# LFSR example ------------------------------------
alice = StreamCipher(key, 'lfsr', poly=[3,1,0]) 
bob   = StreamCipher(key, 'lfsr', poly=[3,1,0])

ciphertext = alice.encrypt(plaintextA) # encryption by Alice
print(f'LFSR Encrypted Ciphertext: {bytes(ciphertext)}')

plaintextB = bob.decrypt(ciphertext) # decryption by Bob
print(f'LFSR Decrypted Plaintext: {bytes(plaintextB)}')

Key: b'12345678'
Plaintext: b'hello world!'

RC4 Encrypted Ciphertext: b'L\xe1&4\xae\x05\xb5\xd1\xe2m!\x9a'
RC4 Decrypted Plaintext: b'hello world!'

ASG Encrypted Ciphertext: b'U\x15!\x87\xd5\xac\xd4\x84\x17\x0c\xc0\t'
ASG Decrypted Plaintext: b'hello world!'

LFSR Encrypted Ciphertext: b'Q\xbd\xb546\xb8\xeerH\x18\x8d\xf2'
LFSR Decrypted Plaintext: b'hello world!'
